In [1]:
import numpy as np
import pandas as pd


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:

df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
## gender adn city are nominal categorical variables (use one hot encoing)
## cough is Ordinal variable (Ordinal Encoder)
## simple imputer on age due to missing values

In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [12]:
## split in test and train before featuring engineering


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [14]:
X_train

,age,gender,fever,cough,city
72,83,Female,101.0,Mild,Kolkata
25,23,Male,NaN,Mild,Mumbai
5,84,Female,NaN,Mild,Bangalore
82,24,Male,98.0,Mild,Kolkata
33,26,Female,98.0,Mild,Kolkata
...,...,...,...,...,...
78,11,Male,100.0,Mild,Bangalore
50,19,Male,101.0,Mild,Delhi
64,42,Male,104.0,Mild,Mumbai
43,22,Female,99.0,Mild,Bangalore


### USUAL WAY OF HANDLING CATEGORICAL COLUMNS ENCODING

In [15]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [16]:
## adding simple imputer to fever col
si=SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [21]:
X_train['cough'].value_counts()

Mild      48
Strong    32
Name: cough, dtype: int64

In [23]:
## Ordinal Encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [24]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [25]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [27]:
## Concatenate all transformed columns for test adn train both
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

### NOW USE COLUMN TRANSFORMER FOR EASE

In [29]:
from sklearn.compose import ColumnTransformer

In [36]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

##remainnder columns that we dont need to apply tranformations on like age here,
## so in order to avoid col. transformer use passthough

#in tuple inside ColumnTransformer 'name of transformer', SimpleImpouter/Ordinal/Onehtoencoding, columns

In [34]:
transformer.fit_transform(X_train).shape

(80, 7)

In [35]:
transformer.transform(X_test).shape


(20, 7)

In [38]:
##https://github.com/campusx-official/100-days-of-machine-learning/blob/main/day28-column-transformer/day28.ipynb